In [1]:
# this folder is used to explore quantities
import re
import pandas as pd
import numpy as np


# cookbook = pd.read_csv('../data/all_ingredients_mapped_to_products_original.csv')
cookbook = pd.read_excel('../data/all_ingredients_mapped_to_products_hand-edited_final.xlsx')
tj_inventory = pd.read_csv("../data/trader_joes_products_v3.csv")

In [2]:
cookbook

,original_text,name,name_confidence,amount,amount_text,unit,amount_confidence,preparation,preparation_confidence,recipe_title,...,likely_sub_category_2,likely_sub_category_2_score,main_category_2,likely_sub_category_3,likely_sub_category_3_score,main_category_3,matched_products,units,matched_categories,match_scores
0,4 tablespoons TJ’s Salted Butter,Salted Butter,0.933367,4.00,4 tablespoons,tablespoon,0.999948,NaN,NaN,Aromatic Garlic Ginger Rice,...,Butter,0.148203,Dairy & Eggs,Cool Desserts,0.145936,From The Freezer,"Butter Quarters, Salted; Cultured Salted Butte...",16 Oz; 8.8 Oz; 4 Oz,Dairy & Eggs; Dairy & Eggs; Dairy & Eggs,0.770; 0.730; 0.726
1,2 tablespoons finely chopped TJ’s Fresh Garlic,Fresh Garlic,0.942706,2.00,2 tablespoons,tablespoon,0.999844,finely chopped,0.961860,Aromatic Garlic Ginger Rice,...,"Beef, Pork & Lamb",0.045175,"Meat, Seafood & Plant-based",Spices,0.037934,For the Pantry,Garlic & Jalapeño Stuffed Olives,17.64 Oz,For the Pantry,0.735
2,1 tablespoon finely chopped TJ’s Fresh Ginger,Fresh Ginger,0.992840,1.00,1 tablespoon,tablespoon,0.999925,finely chopped,0.997657,Aromatic Garlic Ginger Rice,...,Spices,0.021007,For the Pantry,Fruits,0.010147,Fresh Fruits & Veggies,Miso Ginger Broth,32 Fl Oz,For the Pantry,0.724
3,"1 cup TJ’s Jasmine Rice, rinsed",Jasmine Rice,0.997608,1.00,1 cup,cup,0.999978,rinsed,0.998351,Aromatic Garlic Ginger Rice,...,Cereals,0.111725,For the Pantry,Spices,0.051027,For the Pantry,Jasmine Rice From Thailand,3 Lb,For the Pantry,0.951
4,2 cups hot water,hot water,0.995191,2.00,2 cups,cup,0.999965,NaN,NaN,Aromatic Garlic Ginger Rice,...,Salsa & Hot Sauce,0.109329,"Dips, Sauces & Dressings",For Baking & Cooking,0.107880,For the Pantry,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3696,"1 package TJ's Organic Basil, cut into strips",Organic Basil,0.997270,1.00,1 package,package,0.999812,cut into strips,0.999855,Mini Caprese,...,Veggies,0.256747,Fresh Fruits & Veggies,Spices,0.112156,For the Pantry,NaN,NaN,NaN,NaN
3697,1 package TJ's Ciliegine Mozzarella,Ciliegine Mozzarella,0.996692,1.00,1 package,package,0.999895,NaN,NaN,Mini Caprese,...,Cereals,0.079790,For the Pantry,"Beef, Pork & Lamb",0.063148,"Meat, Seafood & Plant-based",NaN,NaN,NaN,NaN
3698,1 bottle TJ's Balsamic Vinaigrette,Balsamic Vinaigrette,0.999404,1.00,1 bottle,bottle,0.999689,NaN,NaN,Mini Caprese,...,Spices,0.063097,For the Pantry,Fruits,0.047781,Fresh Fruits & Veggies,NaN,NaN,NaN,NaN
3699,"1/4 teaspoon TJ's Sea Salt, or to taste",Sea Salt,0.994065,0.25,1/4 teaspoons,teaspoon,0.999956,NaN,NaN,Mini Caprese,...,Spices,0.223477,For the Pantry,Cereals,0.052043,For the Pantry,Sea Salt Crystals; Sea Salt Fine Crystals,3.88 Oz; 26.5 Oz,Spices; Spices,0.727; 0.707


In [3]:
# grabbing amount and units from cookbook to make
ingre_units= cookbook[['amount', 'unit']].copy()
# dropping NaNs in units for now
ingre_units.dropna()
# ingre_units['unit'].unique()

,amount,unit
0,4.00,tablespoon
1,2.00,tablespoon
2,1.00,tablespoon
3,1.00,cup
4,2.00,cup
...,...,...
3696,1.00,package
3697,1.00,package
3698,1.00,bottle
3699,0.25,teaspoon


In [4]:
start_here = [
    "cup",
    "cups",
    "tablespoon",
    "tablespoons",
    "teaspoon",
    "teaspoons",
    "ounce",
    "ounces"]
cup_table_tea_oz = ingre_units[ingre_units['unit'].isin(start_here)].copy()
cup_table_tea_oz

,amount,unit
0,4.00,tablespoon
1,2.00,tablespoon
2,1.00,tablespoon
3,1.00,cup
4,2.00,cup
...,...,...
3688,0.50,teaspoon
3693,2.00,tablespoon
3694,0.50,teaspoon
3699,0.25,teaspoon


In [5]:
tj_inventory.head()

,category,sub_category,product_name,price,unit,url
0,Fresh Fruits & Veggies,Fruits,Sugar Sweet Melon,$3.49,/1 Each,https://www.traderjoes.com/home/products/pdp/s...
1,Fresh Fruits & Veggies,Fruits,Saturn Peaches,$3.99,/19 Oz,https://www.traderjoes.com/home/products/pdp/s...
2,Fresh Fruits & Veggies,Fruits,Fruitful Medley,$3.99,/16 Oz,https://www.traderjoes.com/home/products/pdp/f...
3,Fresh Fruits & Veggies,Fruits,Seedless Lemons,$2.99,/2 Lb,https://www.traderjoes.com/home/products/pdp/s...
4,Fresh Fruits & Veggies,Fruits,Hass Avocado,$1.29,/1 Each,https://www.traderjoes.com/home/products/pdp/h...


In [6]:
tj_inventory["unit"] = tj_inventory["unit"].str.strip().str.replace("/", "", regex=False)
unit_split = tj_inventory["unit"].str.extract(r"(?P<store_amount>[\d\.]+)\s*(?P<store_unit>[A-Za-z\s]+)")
tj_inventory = pd.concat([tj_inventory, unit_split], axis=1)
tj_inventory.head()

,category,sub_category,product_name,price,unit,url,store_amount,store_unit
0,Fresh Fruits & Veggies,Fruits,Sugar Sweet Melon,$3.49,1 Each,https://www.traderjoes.com/home/products/pdp/s...,1,Each
1,Fresh Fruits & Veggies,Fruits,Saturn Peaches,$3.99,19 Oz,https://www.traderjoes.com/home/products/pdp/s...,19,Oz
2,Fresh Fruits & Veggies,Fruits,Fruitful Medley,$3.99,16 Oz,https://www.traderjoes.com/home/products/pdp/f...,16,Oz
3,Fresh Fruits & Veggies,Fruits,Seedless Lemons,$2.99,2 Lb,https://www.traderjoes.com/home/products/pdp/s...,2,Lb
4,Fresh Fruits & Veggies,Fruits,Hass Avocado,$1.29,1 Each,https://www.traderjoes.com/home/products/pdp/h...,1,Each


In [7]:
tj_inventory.columns

Index(['category', 'sub_category', 'product_name', 'price', 'unit', 'url',
       'store_amount', 'store_unit'],
      dtype='object')

In [8]:
tj_units = tj_inventory[['store_amount', 'store_unit']]
# tj_units
tj_units['store_unit'].unique()

array(['Each', 'Oz', 'Lb', 'Fl Oz', 'Doz', 'Pint', 'Qt'], dtype=object)

In [9]:
# focusing just on Oz for now
tj_oz = tj_inventory[tj_inventory["store_unit"].str.lower() == "oz"].copy()
tj_oz.head()

,category,sub_category,product_name,price,unit,url,store_amount,store_unit
1,Fresh Fruits & Veggies,Fruits,Saturn Peaches,$3.99,19 Oz,https://www.traderjoes.com/home/products/pdp/s...,19,Oz
2,Fresh Fruits & Veggies,Fruits,Fruitful Medley,$3.99,16 Oz,https://www.traderjoes.com/home/products/pdp/f...,16,Oz
9,Fresh Fruits & Veggies,Fruits,Cherry Tomatoes on the Vine,$4.99,12 Oz,https://www.traderjoes.com/home/products/pdp/c...,12,Oz
15,Fresh Fruits & Veggies,Fruits,Organic Cranberries,$3.29,12 Oz,https://www.traderjoes.com/home/products/pdp/o...,12,Oz
21,Fresh Fruits & Veggies,Fruits,Organic Strawberries 1 Lb,$6.99,16 Oz,https://www.traderjoes.com/home/products/pdp/o...,16,Oz


In [10]:
# first since most of ingre_units and tj_units are cups, tablespoons, and oz we will focus on those to see what this would look like

unit_to_oz = {
    "cup": 8,
    "cups": 8,
    "tablespoon": 0.5,
    "tablespoons": 0.5,
    "teaspoon": 0.1667,
    "teaspoons": 0.1667,
    "ounce": 1,
    "ounces": 1
}

def convert_to_oz(amount, unit):
    """
    Converts a recipe's quantity + unit to ounces.
    """
    if pd.isna(unit) or unit.lower() not in unit_to_oz:
        return None
    try:
        return float(amount) * unit_to_oz[unit]
    except (TypeError, ValueError):
        return None

In [11]:
cup_table_tea_oz["amount_oz"] = cup_table_tea_oz.apply(
    lambda row: convert_to_oz(row["amount"], row["unit"]), axis=1
)

In [12]:
cup_table_tea_oz

,amount,unit,amount_oz
0,4.00,tablespoon,2.000000
1,2.00,tablespoon,1.000000
2,1.00,tablespoon,0.500000
3,1.00,cup,8.000000
4,2.00,cup,16.000000
...,...,...,...
3688,0.50,teaspoon,0.083350
3693,2.00,tablespoon,1.000000
3694,0.50,teaspoon,0.083350
3699,0.25,teaspoon,0.041675


In [13]:
all_units = ingre_units['unit'].dropna().str.lower().str.strip().unique()

In [14]:
# now lets explore the other things

all_units = ingre_units['unit'].dropna().str.lower().str.strip().unique()
extra_units = [u for u in all_units if u not in unit_to_oz]

extra_counts = (
    ingre_units[ingre_units['unit'].isin(extra_units)]
    ['unit']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'unit', 'unit': 'count'})
)
extra_counts

,count,count
0,package,207
1,bag,144
2,packages,57
3,slices,52
4,can,47
...,...,...
87,oz bottles,1
88,large clove,1
89,picoinch * teaspoon,1
90,batches,1


In [15]:
# this allows us to explore the extra_counts and see if they can match up

def explore_ingredients(unit_name):
    unit_df = cookbook[cookbook['unit'] == unit_name].copy()
    unit_reduced = unit_df[['name', 'amount', 'unit']]
    tj_reduced = tj_inventory[['product_name', 'store_amount', 'store_unit']]
    matches = unit_reduced.merge(tj_reduced, left_on='name', right_on='product_name', how='inner')
    matches['amount'] = pd.to_numeric(matches['amount'], errors='coerce')
    matches['store_amount'] = pd.to_numeric(matches['store_amount'], errors='coerce')
    matches['amount_to_store_amount'] = matches['amount'] * matches['store_amount']
    matches_reduced = matches[['name', 'amount', 'unit','store_amount','amount_to_store_amount', 'store_unit']]
    return matches_reduced

explore_ingredients("pint")

,name,amount,unit,store_amount,amount_to_store_amount,store_unit


In [16]:
# does not work for tablespoon, teaspoon, slices, loaf, cloves, pound, stick, sprig, slice, scoops, bar, pint, heads and the rest under 3 count

In [17]:
# first attempt at merging convert_to_oz and explore_ingredients into one function
def units_to_store_units(df, tj_inventory):
    final_df = df.copy()

    first_units = [
        "cup", "cups",
        "tablespoon", "tablespoons",
        "teaspoon", "teaspoons",
        "ounce", "ounces"
    ]

    does_not_work = [
        "slices", "slice",
        "clove", "cloves",
        "handful", "sprig", "sprigs",
        'loaf', 'pound', 'scoops', 'bar', 'pint',
        'heads'
    ]

    unit_to_oz = {
        "cup": 8, "cups": 8,
        "tablespoon": 0.5, "tablespoons": 0.5,
        "teaspoon": 0.1667, "teaspoons": 0.1667,
        "ounce": 1, "ounces": 1
    }

    def convert_to_oz(amount, unit):
        """Converts a recipe's quantity + unit to ounces."""
        return float(amount) * unit_to_oz[unit]


    def explore_ingredients(unit_name):
        """Merges cookbook ingredients with TJ inventory by name."""
        unit_df = df[df["unit"] == unit_name].copy()
        unit_reduced = unit_df[["name", "amount", "unit"]]
        tj_reduced = tj_inventory[["product_name", "store_amount", "store_unit"]]
        matches = unit_reduced.merge(
            tj_reduced,
            left_on="name",
            right_on="product_name",
            how="inner"
        )
        matches["amount"] = pd.to_numeric(matches["amount"], errors="coerce")
        matches["store_amount"] = pd.to_numeric(matches["store_amount"], errors="coerce")
        matches["amount_to_store"] = matches["amount"] * matches["store_amount"]
        return matches[["name", "amount_to_store", "store_unit"]]

    # handle first_units (direct conversion to Oz)
    mask_first = final_df["unit"].isin(first_units)
    final_df.loc[mask_first, "store_amount"] = final_df.loc[mask_first].apply(
        lambda r: convert_to_oz(r["amount"], r["unit"]), axis=1
    )
    final_df.loc[mask_first, "store_unit"] = "Oz"

    # skip does_not_work
    mask_skip = final_df["unit"].isin(does_not_work)
    final_df.loc[mask_skip, ["store_amount", "store_unit"]] = [None, None]

    # everything else
    mask_other = ~(mask_first | mask_skip)
    unique_units = final_df.loc[mask_other, "unit"].unique()

    for unit_name in unique_units:
        matches = explore_ingredients(unit_name)
        if not matches.empty:
            for _, row in matches.iterrows():
                final_df.loc[
                    (final_df["name"] == row["name"]) & (final_df["unit"] == unit_name),
                    ["store_amount", "store_unit"]
                ] = [row["amount_to_store"], row["store_unit"]]

    return final_df

In [18]:
# final attempt. this takes the df, converts first units with convert_to_oz, then where it can convert using explore_ingredients, if it can't do either it brings over 'amount' and 'unit'
def units_to_store_units2(df, tj_inventory):
    final_df = df.copy()

    first_units = [
        "cup", "cups",
        "tablespoon", "tablespoons",
        "teaspoon", "teaspoons",
        "ounce", "ounces"
    ]

    does_not_work = [
        # "nothing"
        "slices", "slice",
        "clove", "cloves",
        "handful", "sprig", "sprigs",
        "loaf", "pound", "scoops", "bar", "pint",
        "heads"
    ]

    unit_to_oz = {
        "cup": 8, "cups": 8,
        "tablespoon": 0.5, "tablespoons": 0.5,
        "teaspoon": 0.1667, "teaspoons": 0.1667,
        "ounce": 1, "ounces": 1
    }

    def convert_to_oz(amount, unit):
        """Converts a recipe's quantity + unit to ounces."""
        try:
            return float(amount) * unit_to_oz[unit]
        except (TypeError, ValueError, KeyError):
            return None

    def explore_ingredients(unit_name):
        """Merges cookbook ingredients with TJ inventory by name."""
        unit_df = df[df["unit"] == unit_name].copy()
        unit_reduced = unit_df[["name", "amount", "unit"]]
        tj_reduced = tj_inventory[["product_name", "store_amount", "store_unit"]]
        matches = unit_reduced.merge(
            tj_reduced,
            left_on="name",
            right_on="product_name",
            how="inner"
        )
        matches["amount"] = pd.to_numeric(matches["amount"], errors="coerce")
        matches["store_amount"] = pd.to_numeric(matches["store_amount"], errors="coerce")
        matches["amount_to_store"] = matches["amount"] * matches["store_amount"]
        matches.rename(
            columns={
                "amount_to_store": "pantry_amount",
                "store_unit": "pantry_unit"
            },
            inplace=True
        )
        return matches[["name", "pantry_amount", "pantry_unit"]]

    # ---- 1) handle first_units (direct conversion to Oz) ----
    mask_first = final_df["unit"].isin(first_units)
    final_df.loc[mask_first, "pantry_amount"] = final_df.loc[mask_first].apply(
        lambda r: convert_to_oz(r["amount"], r["unit"]), axis=1
    )
    final_df.loc[mask_first, "pantry_unit"] = "Oz"

    # ---- 2) skip does_not_work ----
    mask_skip = final_df["unit"].isin(does_not_work)
    final_df.loc[mask_skip, ["pantry_amount", "pantry_unit"]] = [None, None]

    # ---- 3) everything else ----
    mask_other = ~(mask_first | mask_skip)
    unique_units = final_df.loc[mask_other, "unit"].unique()

    for unit_name in unique_units:
        matches = explore_ingredients(unit_name)
        if not matches.empty:
            for _, row in matches.iterrows():
                final_df.loc[
                    (final_df["name"] == row["name"]) & (final_df["unit"] == unit_name),
                    ["pantry_amount", "pantry_unit"]
                ] = [row["pantry_amount"], row["pantry_unit"]]

    # ---- 4) mark converted_to_store ----
    final_df["converted_to_store"] = (
        final_df["pantry_amount"].notna() & final_df["pantry_unit"].notna()
    )

    # ---- 5) fallback: if not converted, copy original unit + amount ----
    not_converted = ~final_df["converted_to_store"]
    final_df.loc[not_converted, "pantry_amount"] = final_df.loc[not_converted, "amount"]
    final_df.loc[not_converted, "pantry_unit"] = final_df.loc[not_converted, "unit"]

    return final_df

In [19]:
# dropping na since we can't convert if we don't have units and amounts
cookbook2 = cookbook.dropna().copy()

In [25]:
cookbook_revised2 = units_to_store_units2(cookbook2, tj_inventory)
cookbook_revised2

,original_text,name,name_confidence,amount,amount_text,unit,amount_confidence,preparation,preparation_confidence,recipe_title,...,likely_sub_category_3,likely_sub_category_3_score,main_category_3,matched_products,units,matched_categories,match_scores,pantry_amount,pantry_unit,converted_to_store
0,4 tablespoons TJ’s Salted Butter,Salted Butter,0.933367,4.00,4 tablespoons,tablespoon,0.999948,NaN,NaN,Aromatic Garlic Ginger Rice,...,Cool Desserts,0.145936,From The Freezer,"Butter Quarters, Salted; Cultured Salted Butte...",16 Oz; 8.8 Oz; 4 Oz,Dairy & Eggs; Dairy & Eggs; Dairy & Eggs,0.770; 0.730; 0.726,2.000000,Oz,True
1,2 tablespoons finely chopped TJ’s Fresh Garlic,Fresh Garlic,0.942706,2.00,2 tablespoons,tablespoon,0.999844,finely chopped,0.961860,Aromatic Garlic Ginger Rice,...,Spices,0.037934,For the Pantry,Garlic & Jalapeño Stuffed Olives,17.64 Oz,For the Pantry,0.735,1.000000,Oz,True
2,1 tablespoon finely chopped TJ’s Fresh Ginger,Fresh Ginger,0.992840,1.00,1 tablespoon,tablespoon,0.999925,finely chopped,0.997657,Aromatic Garlic Ginger Rice,...,Fruits,0.010147,Fresh Fruits & Veggies,Miso Ginger Broth,32 Fl Oz,For the Pantry,0.724,0.500000,Oz,True
3,"1 cup TJ’s Jasmine Rice, rinsed",Jasmine Rice,0.997608,1.00,1 cup,cup,0.999978,rinsed,0.998351,Aromatic Garlic Ginger Rice,...,Spices,0.051027,For the Pantry,Jasmine Rice From Thailand,3 Lb,For the Pantry,0.951,8.000000,Oz,True
4,2 cups hot water,hot water,0.995191,2.00,2 cups,cup,0.999965,NaN,NaN,Aromatic Garlic Ginger Rice,...,For Baking & Cooking,0.107880,For the Pantry,NaN,NaN,NaN,NaN,16.000000,Oz,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3696,"1 package TJ's Organic Basil, cut into strips",Organic Basil,0.997270,1.00,1 package,package,0.999812,cut into strips,0.999855,Mini Caprese,...,Spices,0.112156,For the Pantry,NaN,NaN,NaN,NaN,1.000000,package,False
3697,1 package TJ's Ciliegine Mozzarella,Ciliegine Mozzarella,0.996692,1.00,1 package,package,0.999895,NaN,NaN,Mini Caprese,...,"Beef, Pork & Lamb",0.063148,"Meat, Seafood & Plant-based",NaN,NaN,NaN,NaN,1.000000,package,False
3698,1 bottle TJ's Balsamic Vinaigrette,Balsamic Vinaigrette,0.999404,1.00,1 bottle,bottle,0.999689,NaN,NaN,Mini Caprese,...,Fruits,0.047781,Fresh Fruits & Veggies,NaN,NaN,NaN,NaN,12.000000,Fl Oz,True
3699,"1/4 teaspoon TJ's Sea Salt, or to taste",Sea Salt,0.994065,0.25,1/4 teaspoons,teaspoon,0.999956,NaN,NaN,Mini Caprese,...,Cereals,0.052043,For the Pantry,Sea Salt Crystals; Sea Salt Fine Crystals,3.88 Oz; 26.5 Oz,Spices; Spices,0.727; 0.707,0.041675,Oz,True


In [21]:
# cookbook2.to_csv("../quantity_exploration/parsed_ingredients_with_pantry_units.csv", index=False)

In [22]:
# the following was used to explore the first iteration of units_to_store_units
unit_nan_pct = cookbook_revised2['unit'].isna().mean() * 100
store_unit_nan_pct = cookbook_revised2['pantry_unit'].isna().mean() * 100

print(f"NaN % in unit: {unit_nan_pct:.1f}%")
print(f"NaN % in store_unit: {store_unit_nan_pct:.1f}%")
print(f"Increase: {store_unit_nan_pct - unit_nan_pct:.1f}%")

NaN % in unit: 0.0%
NaN % in store_unit: 0.0%
Increase: 0.0%


In [23]:
# Find rows that used to have a unit but now lost store_unit
missing_after = cookbook_revised2[cookbook_revised2['unit'].notna() & cookbook_revised2['pantry_unit'].isna()]
print(f"{len(missing_after)} rows lost store_unit after conversion.")
missing_after

0 rows lost store_unit after conversion.


,original_text,name,name_confidence,amount,amount_text,unit,amount_confidence,preparation,preparation_confidence,recipe_title,...,likely_sub_category_3,likely_sub_category_3_score,main_category_3,matched_products,units,matched_categories,match_scores,pantry_amount,pantry_unit,converted_to_store
